# SnapShot and SnapRestore with NetApp DataOps Toolkit

本デモではNetApp DataOps Toolkitを使用して、ONTAPボリューム内のデータを高速にバックアップ・リストアできることを確認します。  
NetApp DataOps Toolkitはオープンソースソフトウェアであり、利用方法も含めて下記GitHub上で公開されています。  
https://github.com/NetApp/netapp-dataops-toolkit/tree/main

## ライブラリのインポート
本デモで使用するNetApp DataOps Toolkitと汎用ライブラリをインポートします。

In [13]:
# NetApp Dataops Toolkitをインポート
from netapp_dataops.traditional import list_volumes, create_snapshot, list_snapshots, restore_snapshot, list_cloud_sync_relationships, sync_cloud_sync_relationship

# 汎用ライブラリ群をインポート
import os
import datetime

## デモ環境の事前確認
NetApp DataOps Toolkitでバックアップを取得する前に、現在のボリュームの状態を確認します。本デモではAmazon FSx for NetApp ONTAP(以下、FSxN)に作成された"non_cache_1"をバックアップの取得対象とします。

In [14]:
# FSxN上に作成されたボリューム一覧の確認
volumes = list_volumes(print_output=True)

Volume Name              Size     Type       NFS Mount Target                       FlexCache    Clone    Source SVM    Source Volume    Source Snapshot
-----------------------  -------  ---------  -------------------------------------  -----------  -------  ------------  ---------------  -----------------
non_cache_1              100.0GB  flexvol    10.104.4.166:/non_cache_1              no           no
flexcache_onprem_to_aws  100.0GB  flexgroup  10.104.4.166:/flexcache_onprem_to_aws  yes          no
flexcache_aws_to_onprem  400.0GB  flexgroup  10.104.4.166:/flexcache_aws_to_onprem  no           no


In [15]:
# バックアップ対象ボリュームが本ノートブックにマウントされていることを確認
os.system("df -h")

Filesystem                             Size  Used Avail Use% Mounted on
devtmpfs                               1.9G     0  1.9G   0% /dev
tmpfs                                  1.9G     0  1.9G   0% /dev/shm
tmpfs                                  1.9G  624K  1.9G   1% /run
tmpfs                                  1.9G     0  1.9G   0% /sys/fs/cgroup
/dev/nvme0n1p1                         180G  135G   46G  75% /
/dev/nvme1n1                           4.8G  348K  4.6G   1% /home/ec2-user/SageMaker
tmpfs                                  386M     0  386M   0% /run/user/1000
tmpfs                                  386M     0  386M   0% /run/user/1001
tmpfs                                  386M     0  386M   0% /run/user/1002
10.104.4.166:/flexcache_onprem_to_aws  380G  4.2G  376G   2% /mnt/fsxn
10.103.4.101:/flexcache_onprem_to_aws  380G  4.2G  376G   2% /mnt/onprem
10.104.4.166:/non_cache_1               95G  9.1G   86G  10% /mnt/fsxn-non-cache


0

In [16]:
# バックアップ対象ボリューム内のファイルを確認
os.system("ls -l /mnt/fsxn-non-cache")

total 24
drwxrwxr-x  3 ec2-user ec2-user 4096 Jul 25 07:07 input
drwxrwxr-x  2 ec2-user ec2-user 4096 Jul 25 07:07 jobs
drwxrwxr-x  2 ec2-user ec2-user 4096 Aug 24 07:57 logs
drwxrwxr-x  3 ec2-user ec2-user 4096 Aug 23 08:45 output
drwxrwxr-x  3 ec2-user ec2-user 4096 Aug  1 09:25 sagemaker
drwxr-xr-x 10 ec2-user ec2-user 4096 Jul 27 07:57 spack


0

## NetApp DataOps Toolkitを使用したボリュームの高速バックアップ
上記で確認したボリュームのバックアップを取得します。下記のコードによりNetApp DataOps ToolkitはONTAPのAPIを介して、ONTAPの機能であるSnapshotを実行します。ONTAPのSnapshot機能では、ボリューム内の実データではなく、データへのポインタ情報のみをバックアップするため大規模なボリュームであっても、バックアップ自体の容量は極めて小さく、バックアップ処理も瞬時に完了します。

In [18]:
# バックアップ対象ボリュームの名称
VOL_NAME = "non_cache_1"

# 取得するSnapshotの名称を生成
t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
now = datetime.datetime.now(JST)
timestamp = now.strftime('%Y%m%d%H%M%S')
SNAPSHOT_NAME = "snapshot_" + timestamp

In [19]:
# NetApp DataOps Toolkitを使用して、ボリュームのSnapshotを取得
create_snapshot(volume_name=VOL_NAME, snapshot_name=SNAPSHOT_NAME, print_output=True)

Creating snapshot 'snapshot_20230824171521'.
Snapshot created successfully.


In [20]:
# 取得したSnapshotの一覧を確認
snap = list_snapshots(volume_name=VOL_NAME, print_output=True)

Snapshot Name            Create Time
-----------------------  -------------------------
snapshot_20230824        2023-08-24 03:19:17+00:00
SNAPSHOT_NAME            2023-08-24 03:31:10+00:00
snapshot_20230824123136  2023-08-24 03:31:36+00:00
snapshot_20230824125359  2023-08-24 03:53:59+00:00
snapshot_20230824125426  2023-08-24 03:54:29+00:00
snapshot_20230824171521  2023-08-24 08:15:23+00:00


## NetApp DataOps Toolkitを使用したボリュームの高速リストア
上記で取得したバックアップをリストアします。NetApp DataOps ToolkitではONTAPのAPIを介して、ONTAPのSnapRestore機能を実行します。ONTAPのSnapRestore機能ではバックアップ時に記録したデータへのポインタ情報のみを復元するため、大規模なボリュームであっても瞬時にリストアを行うことが可能です。

In [21]:
# リストアの確認用にバックアップ対象のボリューム内にテスト用データを作成
os.system("touch /mnt/fsxn-non-cache/demofile")

# demofileが作成されていることを確認
os.system("ls -l /mnt/fsxn-non-cache")

total 24
-rw-rw-r--  1 ec2-user ec2-user    0 Aug 24 08:17 demofile
drwxrwxr-x  3 ec2-user ec2-user 4096 Jul 25 07:07 input
drwxrwxr-x  2 ec2-user ec2-user 4096 Jul 25 07:07 jobs
drwxrwxr-x  2 ec2-user ec2-user 4096 Aug 24 07:57 logs
drwxrwxr-x  3 ec2-user ec2-user 4096 Aug 23 08:45 output
drwxrwxr-x  3 ec2-user ec2-user 4096 Aug  1 09:25 sagemaker
drwxr-xr-x 10 ec2-user ec2-user 4096 Jul 27 07:57 spack


0

In [22]:
# NetApp DataOps Toolkitを使用して、SnapRestoreを実行
restore_snapshot(volume_name=VOL_NAME, snapshot_name=SNAPSHOT_NAME, print_output=True)

Restoring snapshot 'snapshot_20230824171521'.
Snapshot restored successfully.


In [23]:
# 先ほど作成したdemofileが存在しないことを確認
os.system("ls -l /mnt/fsxn-non-cache")

total 24
drwxrwxr-x  3 ec2-user ec2-user 4096 Jul 25 07:07 input
drwxrwxr-x  2 ec2-user ec2-user 4096 Jul 25 07:07 jobs
drwxrwxr-x  2 ec2-user ec2-user 4096 Aug 24 07:57 logs
drwxrwxr-x  3 ec2-user ec2-user 4096 Aug 23 08:45 output
drwxrwxr-x  3 ec2-user ec2-user 4096 Aug  1 09:25 sagemaker
drwxr-xr-x 10 ec2-user ec2-user 4096 Jul 27 07:57 spack


0